In [1]:
from bokeh.charts import Area
from bokeh.io import show, output_notebook
import pandas as pd
import numpy as np
from bokeh.charts.builders.area_builder import AreaBuilder
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
#parse time: parse_dates = [columnName]
d3data = pd.read_csv('d3js.csv', parse_dates=['date'])

#or:
#d3data['date'] = pd.to_datetime(d3data['date'], format = '%m/%d/%y')

# for finding all unique values in a dataframe column:
d3data.key.unique()

dates = d3data.date.unique()

data = {}
for i in d3data.key.unique():
    data[i] = d3data.loc[d3data['key'] == i].iloc[:, 1:]
    
temp = {}
for i in data:
    temp[i] = data[i]['value'].tolist()
    
df = pd.DataFrame(temp, index = dates)

#g0 being the middle line/point when start drawing.
g0 = {'values':[]}
for i in range(len(df)):
    g0['values'].append( - 1/2 * (sum(df.iloc[i])))

#g0points is a series of numbers showing g0 position at each timestamp
g0Points = pd.Series( g0['values'], index = dates)

In [3]:
df.index.append(df.sort_index(ascending = False).index)

data=dict(
    path=[],
    name=[],
    date=[],
    value=[],
)

dates = df.index.append(df.sort_index(ascending = False).index)
cols = df.columns

for i in range(len(df.columns)):
        data['name'].append(cols[i]),
        data['date'].append(dates),
        data['value'].append(df.ix[:, i].values)


for i in range(len(data['name'])):
    if i == 0:
        data['path'].append(np.append(g0Points.values, (g0Points + data['value'][0]).sort_index(ascending = False).values))
    else:
        data['path'].append(np.append(data['path'][i - 1][len(g0Points):][::-1], (data['path'][i - 1][len(g0Points):][::-1] + data['value'][i])[::-1]))


In [4]:
colors = brewer['Blues'][len(cols)]

source = ColumnDataSource(data = data)

#arg x_axis_type = 'datetime' sets default xtick label to datetime format
t = figure(width = 1000, height = 400, x_axis_type = 'datetime', tools = 'pan, wheel_zoom, box_zoom, reset, hover, save')# x_range = test

t.patches('date', 'path', source = source, alpha = 0.5, line_width =1, color = colors)

hover = t.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Genre", "@name"),
    #("date", "@date"),
    #("valueT", "($x, $y)"),
    ("value", "@value"),    
]

show(t)